# Collecting various gridded datasets

- ANU climate data
- MODIS LAI
- Soil moisture from GRAFS

Results are saved to `/g/data/os22/chad_tmp/NEE_modelling/results/input_data/input_data_<YYYY>.nc`

In [ ]:
# !pip install xarray
# !pip install datacube
# !pip install --extra-index-url="https://packages.dea.ga.gov.au" \
#   odc-algo

# !pip install rioxarray
# !pip install dea_tools
# !pip install joblib
# !pip install tqdm
# !pip install geopandas
# !pip install -U scikit-learn
# !pip install dask-ml

In [1]:
from datacube.utils.dask import start_local_dask

import sys
sys.path.append('/g/data/os22/chad_tmp/NEE_modelling/')
from collect_gridded_data import collect_gridded_data

In [2]:
client = start_local_dask(mem_safety_margin='2Gb')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 1
Total threads: 16,Total memory: 44.92 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46071,Workers: 1
Dashboard: /proxy/8787/status,Total threads: 16
Started: Just now,Total memory: 44.92 GiB
Comm: tcp://127.0.0.1:39945,Total threads: 16
Dashboard: /proxy/43927/status,Memory: 44.92 GiB
Nanny: tcp://127.0.0.1:35783,


## Analysis Parameters

In [3]:
time=('2002')

In [4]:
collect_gridded_data(time=time, chunks=dict(latitude=1000, longitude=1000))

   Extracting ANU Climate data
   Extracting MODIS LAI
   Reprojecting datasets
   Adding a lag
   Exporting netcdf


### Try to stich together SIF

In [5]:
import xarray as xr

In [7]:
xr.open_dataarray('/g/data/os22/chad_tmp/NEE_modelling/data/SIF/GOSIF/GOSIF_2000.M03.tif')

<xarray.DataArray 'band_data' (band: 1, y: 3600, x: 7200)>
[25920000 values with dtype=float32]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 -180.0 -179.9 -179.9 -179.8 ... 179.9 179.9 180.0
  * y            (y) float64 89.97 89.92 89.88 89.82 ... -89.88 -89.93 -89.98
    spatial_ref  int64 ...